In [28]:
import pandas as pd
import numpy as np
import time
import os
import sys
import spotipy
import json
import requests
from fuzzywuzzy import fuzz
%precision 4

'%.4f'

In [30]:
# define tracks and spotify API
tracks =pd.read_csv('billboard_2014_to_current.csv')

sp = spotipy.Spotify(
'BQBU0dwy2J6uKbSbEl6Hv-9NOnsoqdJG-Z1DLFH7U2X9AQyThgHlNNlgMr4SgAhAfMe3fJJDVgJw-G9Ur4Iob9Ku0cNv8bAQE0hYB5e1Gft6l-K3qJECfQNJFO0bGxx1k0YRJgb_EibtQR_AhW11ELPNEZIOShZ_ASUk7S0RP6hgJkmx9acj4OULMvOM'
)


In [44]:
'''
Function check if this search get this song from Spotify
'''

def searchSpForInfo(sp,track_name,artist_name):
    res = sp.search(q = track_name)['tracks']['items']
    if res == None or len(res) == 0:
        return (None,None)
    for each in res:
        sp_artists = each['album']['artists']
        for idx,sp_artist in enumerate(sp_artists):
            sp_artist_name = sp_artist['name']
            # fuzzy string matching to check similarity of two names
            if fuzz.token_sort_ratio(sp_artist_name.lower(),artist_name.lower()) > 0.5:
                return (each,idx)
    
    return (None,None)
    

In [56]:
total = 0
not_found = 0
temp = tracks[['artist','title']].drop_duplicates()
tracks['Spotify_ID'] = np.NaN
tracks['Spotify_Popularity'] = np.NaN
tracks['Spotify_Release_Date'] = np.NaN
tracks['Spotify_Artist_Name'] = np.NaN
tracks['Spotify_Artist_ID'] = np.NaN
tracks['Is_Explicit'] = np.NaN
for index, artist,track_title in zip(temp.index, temp.artist,temp.title):
    total += 1
    res,idx = searchSpForInfo(sp,
                          artist_name=artist,
                          track_name=track_title)
    if total % 200 == 0:
        print('Progress {}'.format(total))
    if res is None:
        print(artist)
        print(track_title)
        not_found += 1
        print(not_found / total)
    else:
        album = res['album']
        try:
            artists = album['artists'][idx]
        except IndexError:
            artists = album['artists']
        popularity = res['popularity']
        track_id = res['id']
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title)
                   ,'Spotify_ID'] = track_id
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title)
                   ,'Spotify_Popularity'] = popularity
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title),
                   'Spotify_Release_Date'] = album['release_date']
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title)
                   ,'Spotify_Artist_Name'] = artists['name']
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title)
                   ,'Spotify_Artist_ID'] = artists['id']
        tracks.loc[(tracks.artist == artist) & (tracks.title == track_title)
                   ,'Is_Explicit'] = res['explicit']

Progress 200
Nas Featuring Kanye West
Cop Shot The Kid
0.004310344827586207
XXXTENTACION & Matt Ox
$$$
0.0058823529411764705
Progress 400
21 Savage & Metro Boomin
My Choppa Hate N****s
0.006880733944954129
Progress 600
Progress 800
Kendrick Lamar
Untitled 02 l 06.23.2014.
0.004333694474539545
Kendrick Lamar
Untitled 07 l Levitate
0.005393743257820928
Kendrick Lamar
Untitled 03 l 05.28.2013.
0.006423982869379015
Progress 1000
Progress 1200
Progress 1400
JAY-Z
F.U.T.W.
0.004638833664678595
Kanye West
I Am A God (Featuring God)
0.005256241787122208
Progress 1600
Progress 1800
Tex James Featuring Stuey Rock
Smart Girl (Dumb Booty)
0.0049833887043189366
Aretha Franklin
How Long I've Been Waiting
0.00511770726714432
Rai P & MC Beezy
Swagged Up I Be Killin
0.005583756345177665
Progress 2000
Donell Jones
All About The Sex (It Ain't All About The Sex)
0.005502063273727648
Gena
My Dip In The Club
0.005941499085923218
Progress 2200
Kmelz
Let Me Get Cha Number
0.005998286203941731
Progress 2400
Bu

In [67]:
tracks['acousticness'] = np.NaN
tracks['danceability'] = np.NaN
tracks['energy'] = np.NaN
tracks['instrumentalness'] = np.NaN
tracks['liveness'] = np.NaN
tracks['loudness'] = np.NaN
tracks['mode'] = np.NaN
tracks['speechiness'] = np.NaN
tracks['tempo'] = np.NaN
tracks['valence'] = np.NaN

features = ['acousticness','danceability','energy','instrumentalness',
           'liveness','loudness','mode','speechiness','tempo','valence']

In [68]:
track_ids = tracks.loc[~tracks.Spotify_ID.isna()].Spotify_ID.unique().tolist()
total = len(track_ids)
i = 0
now = time.time()
for track_id in track_ids:
    i += 1
    if i % 300 == 0:
        print('Progress %4.2f' % (i / total * 100))
        print('Takes {} seconds'.format(time.time() - now))
    ad_features = sp.audio_features(tracks=[track_id])[0]
    if ad_features == None:
        continue
    for each in features:
        tracks.loc[tracks.Spotify_ID == track_id,each] = ad_features[each]


Progress 12.47
Takes 36.51756691932678 seconds
Progress 24.95
Takes 83.90391993522644 seconds
Progress 37.42
Takes 129.07318091392517 seconds
Progress 49.90
Takes 172.47104597091675 seconds
Progress 62.37
Takes 215.84296202659607 seconds
Progress 74.84
Takes 257.37428092956543 seconds
Progress 87.32
Takes 305.2892129421234 seconds
Progress 99.79
Takes 351.0020010471344 seconds


In [71]:
tracks.to_csv('Billboard_with_Spotify_features.csv',index = False)

In [70]:
tracks.acousticness.isna().sum()

103

In [76]:
len(tracks.loc[tracks.Spotify_ID.isna(),'title'].unique())

21

In [72]:
tracks.describe()

,lastPos,peakPos,rank,weeks,Spotify_Popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
count,32150.000000,32150.000000,32150.000000,32150.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000,32047.000000
mean,32.237014,31.221244,38.174961,11.992473,64.085499,0.183763,0.674142,0.625148,0.012212,0.184233,-6.701170,0.604799,0.140477,122.692344,0.454977
std,26.546259,26.103753,26.102962,9.867816,16.667355,0.222850,0.145886,0.168337,0.085571,0.140030,2.717399,0.488901,0.118926,30.249451,0.213188
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-50.319000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,9.000000,17.000000,4.000000,56.000000,0.023700,0.582000,0.521000,0.000000,0.098000,-7.980000,0.000000,0.047000,97.448000,0.287000
50%,28.000000,25.000000,34.000000,10.000000,67.000000,0.093000,0.689000,0.632000,0.000000,0.126000,-6.312000,1.000000,0.093200,124.719000,0.442000
75%,47.000000,47.000000,51.000000,17.000000,76.000000,0.258500,0.776000,0.743000,0.000013,0.229000,-4.985000,1.000000,0.212000,142.135000,0.607000
max,100.000000,100.000000,100.000000,74.000000,98.000000,0.995000,0.981000,0.997000,0.980000,0.942000,0.175000,1.000000,0.888000,208.021000,0.978000
